<a href="https://colab.research.google.com/github/Noppadol35/IS/blob/main/Feed-Forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
from google.colab import files

# อัปโหลดไฟล์
uploaded = files.upload()

# ดึงชื่อไฟล์ที่อัปโหลด
filename = list(uploaded.keys())[0]

# โหลด dataset
df = pd.read_csv(filename)

# แสดงข้อมูลตัวอย่าง
df.head()


Saving Dataset (1).csv to Dataset (1) (5).csv


,gender,age,hypertension,heart_disease,Marriage,work_type,Living_type,avg_glucose,bmi,smoking_status,illness
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np

# คัดแยก features และ target
X = df.drop(columns=['illness'])
y = df['illness']

# แปลง categorical features เป็นตัวเลข
categorical_cols = ['gender', 'Marriage', 'work_type', 'Living_type', 'smoking_status']
X = pd.get_dummies(X, columns=categorical_cols)

# เติมค่าที่หายไป (ถ้ามี)
X = X.fillna(X.mean())

# มาตรฐานข้อมูลตัวเลข
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)

# แยก train/test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ตรวจสอบขนาดของชุดข้อมูลที่เตรียมเสร็จ
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((320, 19), (80, 19), (320,), (80,))

In [107]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import AdamW

# ปรับโครงสร้างโมเดลให้ดีขึ้น
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),

    Dense(64),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),

    Dense(32),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])

# ใช้ AdamW optimizer พร้อม learning rate ที่ลดลง
model.compile(optimizer=AdamW(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# เทรนโมเดลใหม่
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# ประเมินผลลัพธ์บน test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # แปลงค่าความน่าจะเป็นเป็น 0 หรือ 1

test_accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - accuracy: 0.5304 - loss: 0.7766 - val_accuracy: 0.5500 - val_loss: 0.6568
Epoch 2/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5935 - loss: 0.6944 - val_accuracy: 0.6125 - val_loss: 0.6254
Epoch 3/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6194 - loss: 0.6331 - val_accuracy: 0.6750 - val_loss: 0.6016
Epoch 4/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5697 - loss: 0.6747 - val_accuracy: 0.7125 - val_loss: 0.5880
Epoch 5/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6889 - loss: 0.5791 - val_accuracy: 0.7250 - val_loss: 0.5769
Epoch 6/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6829 - loss: 0.5639 - val_accuracy: 0.7375 - val_loss: 0.5674
Epoch 7/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7251 - loss: 0.5306 - val_accuracy: 0.7375 - val_loss: 0.5606
Epoch 8/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7569 - loss: 0.5033 - val_accuracy: 0.

In [108]:
# แสดงผลลัพธ์
print("\n--- ผลลัพธ์การประเมินโมเดล ---")
print(f"Accuracy บน train set: {history.history['accuracy'][-1]:.2%}")
print(f"Accuracy บน test set: {test_accuracy:.2%}")

# แสดงโครงสร้างของโมเดล
model.summary()

# นับจำนวน layers และ neurons
num_layers = len(model.layers)
neurons_per_layer = [layer.units for layer in model.layers if hasattr(layer, 'units')]

# จำนวน epochs ที่ใช้ในการฝึก
num_epochs = len(history.epoch)

# คำนวณจำนวน iterations ต่อ epoch (จำนวน batch)
batch_size = 32
num_iterations_per_epoch = len(X_train) // batch_size

# แสดงผลลัพธ์
print("\n--- สรุปผลการออกแบบโมเดล ---")
print(f"1. จำนวน layers ที่ออกแบบ: {num_layers}")
print(f"2. จำนวน neurons ในแต่ละ layer: {neurons_per_layer}")
print(f"3. จำนวน epochs ที่ใช้ฝึก: {num_epochs}")
print(f"   จำนวน iterations ต่อ epoch: {num_iterations_per_epoch}")



--- ผลลัพธ์การประเมินโมเดล ---
Accuracy บน train set: 90.31%
Accuracy บน test set: 80.00%


Model: "sequential_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_204 (Dense)                    │ (None, 256)                 │           5,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_48               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_84 (LeakyReLU)           │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_84 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_205 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_49               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_85 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_85 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_206 (Dense)                    │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_50               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_86 (LeakyReLU)           │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_86 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_207 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_51               │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_87 (LeakyReLU)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_87 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_208 (Dense)                    │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 148,997 (582.02 KB)

 Trainable params: 49,345 (192.75 KB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 98,692 (385.52 KB)


--- สรุปผลการออกแบบโมเดล ---
1. จำนวน layers ที่ออกแบบ: 17
2. จำนวน neurons ในแต่ละ layer: [256, 128, 64, 32, 1]
3. จำนวน epochs ที่ใช้ฝึก: 150
   จำนวน iterations ต่อ epoch: 10
